<h1>Assignment for the Case AML (System Development for Fintech)</h1>
<h4>David Widlak</h4>
This document represents an anti money laundering algorithm. This document converts MT103 Swift messages into a workable data structure. These messages are then analyzed using different functions and suspicious transactions are flagged.

<h4>Initial inclusion of assisting Python libraries</h4>

In [2433]:
import re # Regex functionality
import pandas as pd # Facilitates data operations
import numpy as np # Used to impute NaN values into dataframes
import pycountry # Used to associate full country names with (ISO 3166-1) alpha-2 codes

<h4>Converting MT103 Swift message into a different data structure</h4>

In [2434]:
# Function to disect currency, amounth and date from MT103 tag 32A
def extract_32a(value):
    date = value[:6]
    # Extract the currency type (next 3 characters)
    currency_type = value[6:9]
    # Extract the amount (rest of the string)
    amount = value[9:]

    return date, currency_type, amount

In [2435]:
# Function to structurize MT103 messages into a dictionary
def structurize_mt103(message):
    msg_dic = {}
    # In case the message is a text file
    message = message.read()
    # Split the unstructured message with regex
    key_value = re.split(r'}{|\n:', message)
    # Key without associating value
    if '4:' in key_value:
        key_value.remove('4:')
    elif '{4:' in key_value:
        key_value.remove('{4:')
    elif '4' in key_value:
        key_value.remove('4')

    for kv_str in key_value:
        # Split into key value pairs
        key, value = kv_str.split(':', maxsplit = 1)
        msg_dic[key] = value

    return msg_dic

In [2436]:
# Create dictionaries from message files 
with open('mt103-1.txt', 'r') as file:
    swift_dic_1 = structurize_mt103(file)

with open('mt103-2.txt', 'r') as file:
    swift_dic_2 = structurize_mt103(file)

with open('mt103-3.txt', 'r') as file:
    swift_dic_3 = structurize_mt103(file)

In [2437]:
# Dictionary used to associate swift tag with column name. Will be used in functions below
swift_corr = {
    'transaction_date': ['32A', 'date'],
    'transaction_id': ['20', 'string'],
    'transaction_message': [['71G', '70'], 'string', ''],
    'transaction_currency': ['32A', ['regex', "^(AED|AFN|ALL|AMD|ANG|AOA|ARS|AUD|AWG|AZN|BAM|BBD|BDT|BGN|BHD|B IF|BMD|BND|BOB|BOV|BRL|BSD|BTN|BWP|BYR|BZD|CAD|CDF|CHE|CHF|CHW|CLF|CLP |CNY|COP|COU|CRC|CUC|CUP|CVE|CZK|DJF|DKK|DOP|DZD|EGP|ERN|ETB|EUR|FJD|F KP|GBP|GEL|GHS|GIP|GMD|GNF|GTQ|GYD|HKD|HNL|HRK|HTG|HUF|IDR|ILS|INR|IQD |IRR|ISK|JMD|JOD|JPY|KES|KGS|KHR|KMF|KPW|KRW|KWD|KYD|KZT|LAK|LBP|LKR|L RD|LSL|LTL|LVL|LYD|MAD|MDL|MGA|MKD|MMK|MNT|MOP|MRO|MUR|MVR|MWK|MXN|MXV |MYR|MZN|NAD|NGN|NIO|NOK|NPR|NZD|OMR|PAB|PEN|PGK|PHP|PKR|PLN|PYG|QAR|R ON|RSD|RUB|RWF|SAR|SBD|SCR|SDG|SEK|SGD|SHP|SLL|SOS|SRD|SSP|STD|SVC|SYP |SZL|THB|TJS|TMT|TND|TOP|TRY|TTD|TWD|TZS|UAH|UGX|USD|USN|USS|UYI|UYU|U ZS|VEF|VND|VUV|WST|XAF|XAG|XAU|XBA|XBB|XBC|XBD|XCD|XDR|XFU|XOF|XPD|XPF |XPT|XSU|XTS|XUA|XXX|YER|ZAR|ZMW|ZWL)$"]],
    'transaction_amount': ['32A', 'string'],
    'transaction_type': ['23B', 'string'],
    'transaction_direction': ['null', ['i', 'o', 'io']],
    'transaction_status': ['null', ['accepted', 'rejected']],
    'instrument_type': ['23B', ['cash','check','ach/lcy_transfers','wire','securities','e- money/mobile_money','travellers_cheques','prepaid_cards','certified_cheques','vouchers','cashier_cheques/money_order','precious_metal','cryp to/virtual_assets','interest/dividend','other']],
    'originator_full_name': ['50', 'string'],
    'originator_first_name': ['null', ['string', '']],
    'originator_middle_names_patronymic': ['null', ['string', '']],
    'originator_last_name': ['null', ['string', '']],
    'originator_address': ['50', 'string'],
    'originator_country': ['50', ['regex', "^(AF|AX|AL|DZ|AS|AD|AO|AI|AQ|AG|AR|AM|AW|AU|AT|AZ|BS|BH|BD|BB|B Y|BE|BZ|BJ|BM|BT|BO|BQ|BA|BW|BV|BR|IO|BN|BG|BF|BI|KH|CM|CA|CV|KY|CF|TD |CL|CN|CX|CC|CO|KM|CG|CD|CK|CR|CI|HR|CU|CW|CY|CZ|DK|DJ|DM|DO|EC|EG|SV| GQ|ER|EE|ET|FK|FO|FJ|FI|FR|GF|PF|TF|GA|GM|GE|DE|GH|GI|GR|GL|GD|GP|GU|G T|GG|GN|GW|GY|HT|HM|VA|HN|HK|HU|IS|IN|ID|IR|IQ|IE|IM|IL|IT|JM|JP|JE|JO |KZ|KE|KI|KP|KR|KW|KG|LA|LV|LB|LS|LR|LY|LI|LT|LU|MO|MK|MG|MW|MY|MV|ML| MT|MH|MQ|MR|MU|YT|MX|FM|MD|MC|MN|ME|MS|MA|MZ|MM|NA|NR|NP|NL|NC|NZ|NI|N E|NG|NU|NF|MP|NO|OM|PK|PW|PS|PA|PG|PY|PE|PH|PN|PL|PT|PR|QA|RE|RO|RU|RW |BL|SH|KN|LC|MF|PM|VC|WS|SM|ST|SA|SN|RS|SC|SL|SG|SX|SK|SI|SB|SO|ZA|GS| SS|ES|LK|SD|SR|SJ|SZ|SE|CH|SY|TW|TJ|TZ|TH|TL|TG|TK|TO|TT|TN|TR|TM|TC|T V|UG|UA|AE|GB|US|UM|UY|UZ|VU|VE|VN|VG|VI|WF|EH|YE|ZM|ZW)$"]],
    'originator_account_number': ['50', 'string'],
    'originator_branch_id': ['null', ['string', '']],
    'originator_bic': ['50', 'string'],
    'originator_fi_name': ['50', 'string'],
    'originator_fi_country': ['50', 'string'],
    'incoming_intermediary_fi_bic': ['56A', ['regex', r"/([a-z]{4}[a-z]{2}[a-z0-9]{2}([a- z0-9]{3})?;?)+/gmi", '']],
    'outgoing_intermediary_fi_bic': ['57A', ['regex', r"/([a-z]{4}[a-z]{2}[a-z0-9]{2}([a- z0-9]{3})?;?)+/gmi", '']],
    'beneficiary_full_name': ['59', 'string'],
    'beneficiary_first_name': ['null', ['string', '']],
    'beneficiary_middle_names_patronymic': ['null', ['string', '']],
    'originator_middle_names_patronymic': ['null', ['string', '']],
    'beneficiary_last_name': ['null', ['string', '']],
    'beneficiary_account_number': ['59', 'string'],
    'beneficiary_address': ['59', 'string'],
    'beneficiary_country': ['59', ['regex', "^(AF|AX|AL|DZ|AS|AD|AO|AI|AQ|AG|AR|AM|AW|AU|AT|AZ|BS|BH|BD|BB|B Y|BE|BZ|BJ|BM|BT|BO|BQ|BA|BW|BV|BR|IO|BN|BG|BF|BI|KH|CM|CA|CV|KY|CF|TD |CL|CN|CX|CC|CO|KM|CG|CD|CK|CR|CI|HR|CU|CW|CY|CZ|DK|DJ|DM|DO|EC|EG|SV| GQ|ER|EE|ET|FK|FO|FJ|FI|FR|GF|PF|TF|GA|GM|GE|DE|GH|GI|GR|GL|GD|GP|GU|G T|GG|GN|GW|GY|HT|HM|VA|HN|HK|HU|IS|IN|ID|IR|IQ|IE|IM|IL|IT|JM|JP|JE|JO |KZ|KE|KI|KP|KR|KW|KG|LA|LV|LB|LS|LR|LY|LI|LT|LU|MO|MK|MG|MW|MY|MV|ML| MT|MH|MQ|MR|MU|YT|MX|FM|MD|MC|MN|ME|MS|MA|MZ|MM|NA|NR|NP|NL|NC|NZ|NI|N E|NG|NU|NF|MP|NO|OM|PK|PW|PS|PA|PG|PY|PE|PH|PN|PL|PT|PR|QA|RE|RO|RU|RW |BL|SH|KN|LC|MF|PM|VC|WS|SM|ST|SA|SN|RS|SC|SL|SG|SX|SK|SI|SB|SO|ZA|GS| SS|ES|LK|SD|SR|SJ|SZ|SE|CH|SY|TW|TJ|TZ|TH|TL|TG|TK|TO|TT|TN|TR|TM|TC|T V|UG|UA|AE|GB|US|UM|UY|UZ|VU|VE|VN|VG|VI|WF|EH|YE|ZM|ZW)$"]],
    'beneficiary_branch_id': ['null', ['string', '']],
    'beneficiary_bic': ['59', 'string'],
    'beneficiary_fi_name': ['59', 'string'],
    'beneficiary_fi_country': ['59', 'string'],
}


In [2438]:
# Takes a complete bank account string from a MT103 message and dynamically splits the data
def extract_bank_information(value):
    lines = value.split('\n')

    bic_or_account = None
    name = None
    is_bic = None
    bic = None

    # Scan for BIC, to differentiate between BIC and account number
    for idx, line in enumerate(lines):
        if re.match(r'(?<![A-Za-z0-9/])(/?[A-Za-z]{6}[A-Za-z0-9]{2}([A-Za-z0-9]{3})?)(?![A-Za-z0-9])', line):
            if bic_or_account is None:
                bic_or_account = line
                name = lines[idx + 1]
                location = lines[idx + 2:]
                is_bic = True
                break
            else:
                bic = line
                break

    if not bic_or_account:
        # Find the line with the account number
        for idx, line in enumerate(lines):
            if line.startswith('/'):
                bic_or_account = line[1:]
                lines.pop(idx)
                break
                
        name = lines[0]
        is_bic = False
        location = lines[1:]

    if not bic:
        if str(lines[-1]).startswith('/'):
            bic = lines[-1][1:]
            location.pop()

    if location and ',' in location[-1]:
        sub_list = location[-1].split(',')
        location.pop()
        location.extend(sub_list)

    city_or_address = ' '.join(location[:-1]) if location else ''
    is_address = any(char.isdigit() for char in city_or_address)
    country = location[-1] if location else ''

    return bic_or_account, is_bic, name, city_or_address, is_address, country, bic

swift = '''/NL20ABNA0404875234
ABNANL2A
ABC SUPPLIERS BV
AMSTERDAM, NETHERLANDS'''

extract_bank_information(swift)
print(extract_bank_information(swift))

('ABNANL2A', True, 'ABC SUPPLIERS BV', 'AMSTERDAM', False, ' NETHERLANDS', None)


In [2439]:
# Function to populate the dataframe from MT103 dictionary above. Function takes swift MT103 messages (list) as parameter
# and uses the swift_corr as translator, it gaps the bridge between swift tags and readable variable names and the values.
def populate_dataframe(mt103_messages, df):
    new_rows = []

    # Loop through the messages
    for mt103_message in mt103_messages:
        new_row = {}

        # Bind each item in the dictionary with a value in the swift_corr
        for item in swift_corr:
            for key in mt103_message:
                # Check if the first value of the key-value pair is a list or singular value
                if isinstance(swift_corr[item][0], list):
                    for corr_key in swift_corr[item][0]:
                        if corr_key in key:
                            try:
                                # Insert each value as a string
                                new_row[item] = str(mt103_message[key])
                            except:
                                print('Cannot convert data to string type')
                else:
                    if swift_corr[item][0] in key:
                        try:
                            # Insert each value as a string
                            new_row[item] = str(mt103_message[key])
                        except:
                            print('Cannot convert data to string type')
        new_rows.append(new_row)

    # Append the new rows to the existing DataFrame
    return pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

# Create an empty dataframe with the required columns
columns = list(swift_corr.keys())
df = pd.DataFrame(columns=columns)

# Call the function with the list of MT103 message dictionaries and the existing DataFrame
mt103_message_list = [swift_dic_1, swift_dic_2, swift_dic_3]  # List of dictionaries representing MT103 messages
df = populate_dataframe(mt103_message_list, df)
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,transaction_direction,transaction_status,instrument_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_account_number,beneficiary_address,beneficiary_country,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country
0,"210322USD5000,",MT103 0001,/INS/THIS IS A PAYMENT FOR TUNA SUPPLY\n-},"210322USD5000,","210322USD5000,",CRED,NaN,NaN,CRED,/DE98765432101234567890\nCOMMERZBANK AG\nHAMBU...,NaN,NaN,NaN,/DE98765432101234567890\nCOMMERZBANK AG\nHAMBU...,/DE98765432101234567890\nCOMMERZBANK AG\nHAMBU...,/DE98765432101234567890\nCOMMERZBANK AG\nHAMBU...,NaN,/DE98765432101234567890\nCOMMERZBANK AG\nHAMBU...,/DE98765432101234567890\nCOMMERZBANK AG\nHAMBU...,/DE98765432101234567890\nCOMMERZBANK AG\nHAMBU...,NaN,"/HBUKGB4BXXX\nHSBC BANK PLC\nLONDON, UNITED KI...",/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,NaN,NaN,NaN,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,NaN,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...
1,"210322USD10000,",MT103 0001,/MSG/PAYMENT FOR GOODS\n-},"210322USD10000,","210322USD10000,",CRED,NaN,NaN,CRED,/US12345678901234567890\nABC INDUSTRIES\n123 M...,NaN,NaN,NaN,/US12345678901234567890\nABC INDUSTRIES\n123 M...,/US12345678901234567890\nABC INDUSTRIES\n123 M...,/US12345678901234567890\nABC INDUSTRIES\n123 M...,NaN,/US12345678901234567890\nABC INDUSTRIES\n123 M...,/US12345678901234567890\nABC INDUSTRIES\n123 M...,/US12345678901234567890\nABC INDUSTRIES\n123 M...,/ICBKCNBJGZU\nINDUSTRIAL AND COMMERCIAL BANK O...,/CITIUS33\nCITIBANK NA\n111 WALL STREET\nNEW Y...,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,NaN,NaN,NaN,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,NaN,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...
2,"210322USD9899,",MT103 0001,INV NO. 12345\nREF. 98765\nSUPPLY OF GOODS AS ...,"210322USD9899,","210322USD9899,",CRED,NaN,NaN,CRED,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,NaN,NaN,NaN,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,NaN,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,"/SCBLGB2LXXX\nSTANDARD CHARTERED BANK\nLONDON, UK","/BNYMUS33XXX\nBNY MELLON\nNEW YORK, NY, US",/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,NaN,NaN,NaN,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,NaN,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...


In [2440]:
# Universal function for cleaning up strings. Takes the DF, column and (list) of strings which needs to be deleted
def clean_strings_in_df(df, column_name, sub_strings):
    for i in range(len(df)):
        cell_value = df.loc[i, column_name]
        for sub_string in sub_strings:
            if sub_string in cell_value:
                cell_value = cell_value.replace(sub_string, "")
                df.loc[i, column_name] = cell_value

In [2442]:
# Universal function for extracting multiline bank data
def update_df_with_bank_info(df, bank_info, columns, info_type, row_idx):
    # Split the bank_info list into multiple variables 
    bic_or_account, is_bic, name, city_or_address, is_address, country, bic = bank_info

    # Inpute the data, info_type can be 'originator' or 'beneficiary'.
    # This makes the function reusable for both parties
    for item in columns:
        if f'{info_type}_account_number' in item or f'{info_type}_bic' in item:
            if is_bic and f'{info_type}_bic' in item:
                df.at[row_idx, item] = bic_or_account
            # TODO: Make sure the BIC is also filled in for bic_or_account num if is_bic = true
            elif not is_bic:
                if f'{info_type}_bic' in item and bic:
                    df.at[row_idx, item] = bic
                elif f'{info_type}_account_number' in item:
                    df.at[row_idx, item] = bic_or_account
        elif f'{info_type}_fi_country' in item and is_bic:
            df.at[row_idx, item] = bic_or_account[4:6]
        elif f'{info_type}_fi_country' in item and not is_bic:
            df.at[row_idx, item] = bic[4:6]
        elif f'{info_type}_branch' in item or f'{info_type}_fi_name' in item or f'{info_type}_full_name' in item:
            df.at[row_idx, item] = name
        elif f'{info_type}_address' in item:
            df.at[row_idx, item] = city_or_address
        elif f'{info_type}_country' in item:
            if country:
                # Find the alpha 2 country code by full name using Pycountry. 
                if pycountry.countries.search_fuzzy(country)[0].alpha_2:
                    country_iso = pycountry.countries.search_fuzzy(country)[0].alpha_2
                    df.at[row_idx, item] = country_iso
                else:
                    df.at[row_idx, item] = country

def impute_and_clean_df(df):
    # Ensure string variables
    df.loc[:, :] = df.astype(str)

    for row_idx, row in df.iterrows():
        # Create list with keys that match 32A tag in swift_corr dictionary 
        transaction_columns = [key for key in swift_corr if '32A' in swift_corr[key]]
        # Get the raw full transaction info string
        transaction_info_str = row[transaction_columns[0]]
        # Seperate the string into variables
        trans_date, trans_currency, trans_amount = extract_32a(transaction_info_str)
        # Split transaction data and impute
        for item in transaction_columns:
            if 'transaction_date' in item:
                df.at[row_idx, item] = trans_date
            elif 'transaction_amount' in item:
                df.at[row_idx, item] = trans_amount
            elif 'transaction_currency' in item:
                df.at[row_idx, item] = trans_currency

        # For each of the bank related tags, get the keys, extract the data, and use the function above to impute values
        bank_originator_columns = [key for key in swift_corr if any(x in swift_corr[key] for x in ['50', '50K', '71G', '50A'])]
        originator_info_str = row['originator_address']
        print(extract_bank_information(originator_info_str))
        originator_bank_info = extract_bank_information(originator_info_str)
        update_df_with_bank_info(df, originator_bank_info, bank_originator_columns, 'originator', row_idx)

        # Same as the lines above, but for beneficiary
        bank_beneficiary_columns = [key for key in swift_corr if '59' in swift_corr[key]]
        beneficiary_info_str = row['beneficiary_address']
        beneficiary_bank_info = extract_bank_information(beneficiary_info_str)
        update_df_with_bank_info(df, beneficiary_bank_info, bank_beneficiary_columns, 'beneficiary', row_idx)

        # Clean cell data 
        clean_strings_in_df(df, 'originator_bic', ['/BIC/', '/'])
        clean_strings_in_df(df, 'transaction_message', [',', '/INS/', '/MSG/', '\n-}', '\n'])
        clean_strings_in_df(df, 'transaction_amount', [','])

impute_and_clean_df(df)
pd.set_option('display.max_columns', None)
df

IndexError: list index out of range

In [ ]:
# Transaction direction are indicated by the MT message type
# The message type can be extracted from the transaction ID
def add_transaction_direction(df):
    for i in range(len(df)):
        cell_trans_id = df.loc[i, 'transaction_id']
        if 'MT103' in cell_trans_id:
            df.loc[i, 'transaction_direction'] = 'o'
        if 'MT202' in cell_trans_id:
            df.loc[i, 'transaction_direction'] = 'o'
        if 'MT300' in cell_trans_id:
            df.loc[i, 'transaction_direction'] = 'i'
        if 'MT910' in cell_trans_id:
            df.loc[i, 'transaction_direction'] = 'i'
        if 'MT950' in cell_trans_id:
            df.loc[i, 'transaction_direction'] = 'io'

add_transaction_direction(df)
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,transaction_direction,transaction_status,instrument_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_account_number,beneficiary_address,beneficiary_country,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country
0,210322,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,o,nan,CRED,COMMERZBANK AG,nan,nan,nan,HAMBURG,DE,DE98765432101234567890,nan,COBADEHHXXX,COMMERZBANK AG,DE,nan,"/HBUKGB4BXXX\nHSBC BANK PLC\nLONDON, UNITED KI...",NORDFISCH GMBH,nan,nan,nan,GB57METR12345678901234,BODENSEE STR. 226 22761 HAMBURG,DE,nan,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,NORDFISCH GMBH,
1,210322,MT103 0001,PAYMENT FOR GOODS,USD,10000,CRED,o,nan,CRED,ABC INDUSTRIES,nan,nan,nan,"123 MAIN STREET NEW YORK, NY 10001",US,US12345678901234567890,nan,ABCBUS33XXX,ABC INDUSTRIES,US,/ICBKCNBJGZU\nINDUSTRIAL AND COMMERCIAL BANK O...,/CITIUS33\nCITIBANK NA\n111 WALL STREET\nNEW Y...,XYZ SUPPLIERS,nan,nan,nan,CN123456789012345678,123 HUANGPU ROAD SHANGHAI,CN,nan,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,XYZ SUPPLIERS,
2,210322,MT103 0001,INV NO. 12345REF. 98765SUPPLY OF GOODS AS PER ...,USD,9899,CRED,o,nan,CRED,ABC SUPPLIERS BV,nan,nan,nan,AMSTERDAM,NL,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,nan,ABNANL2A,ABC SUPPLIERS BV,NL,"/SCBLGB2LXXX\nSTANDARD CHARTERED BANK\nLONDON, UK","/BNYMUS33XXX\nBNY MELLON\nNEW YORK, NY, US",AFRICAN EXPORT-IMPORT BANK,nan,nan,nan,PASSNGLAXXX,"LAGOS, NIGERIA XYZ ENTERPRISES LTD LAGOS",NG,nan,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,AFRICAN EXPORT-IMPORT BANK,


In [ ]:
# Function to extract BIC codes from intermediary banks
def extract_bic_intermediary(df):
    # Find columns that contain intermediary bic info
    cols = [col for col in df.columns if 'intermediary_fi_bic' in col]
    for col in cols:
        # Split the string based on '\n', keep the first item which is the bic
        raw_bic = df[col].str.split('\n').str[0]
        raw_bic = str(raw_bic[0]).replace('/', '')
        df[col] = raw_bic

extract_bic_intermediary(df)
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,transaction_direction,transaction_status,instrument_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_account_number,beneficiary_address,beneficiary_country,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country
0,210322,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,o,nan,CRED,COMMERZBANK AG,nan,nan,nan,HAMBURG,DE,DE98765432101234567890,nan,COBADEHHXXX,COMMERZBANK AG,DE,nan,HBUKGB4BXXX,NORDFISCH GMBH,nan,nan,nan,GB57METR12345678901234,BODENSEE STR. 226 22761 HAMBURG,DE,nan,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,NORDFISCH GMBH,
1,210322,MT103 0001,PAYMENT FOR GOODS,USD,10000,CRED,o,nan,CRED,ABC INDUSTRIES,nan,nan,nan,"123 MAIN STREET NEW YORK, NY 10001",US,US12345678901234567890,nan,ABCBUS33XXX,ABC INDUSTRIES,US,nan,HBUKGB4BXXX,XYZ SUPPLIERS,nan,nan,nan,CN123456789012345678,123 HUANGPU ROAD SHANGHAI,CN,nan,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,XYZ SUPPLIERS,
2,210322,MT103 0001,INV NO. 12345REF. 98765SUPPLY OF GOODS AS PER ...,USD,9899,CRED,o,nan,CRED,ABC SUPPLIERS BV,nan,nan,nan,AMSTERDAM,NL,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,nan,ABNANL2A,ABC SUPPLIERS BV,NL,nan,HBUKGB4BXXX,AFRICAN EXPORT-IMPORT BANK,nan,nan,nan,PASSNGLAXXX,"LAGOS, NIGERIA XYZ ENTERPRISES LTD LAGOS",NG,nan,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,AFRICAN EXPORT-IMPORT BANK,


<h1>AML Functions</h1>
<h3>Round amounth payments</h3>
Looking at round payment transaction amounts is important for anti-money laundering (AML) because it can be an indicator of suspicious activity that may warrant further investigation.

Detection is done by looking if the amount can be divided by 100, if so, the transaction gets flagged as round_number = True.

In [ ]:
def is_round(df):
    # Try to convert into a number, if possible throw exception
    for index, num in df['transaction_amount'].iteritems():
        try:
            num = int(num)
            # Check for round number and add to column
            if num % 100 == 0:
                df.at[index, 'round_number'] = True
            else:
                df.at[index, 'round_number'] = False
        except ValueError:
            return "Could not convert to number"
    
is_round(df)
df


,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,transaction_direction,transaction_status,instrument_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_account_number,beneficiary_address,beneficiary_country,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country,round_number
0,210322,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,o,nan,CRED,COMMERZBANK AG,nan,nan,nan,HAMBURG,DE,DE98765432101234567890,nan,COBADEHHXXX,COMMERZBANK AG,DE,nan,HBUKGB4BXXX,NORDFISCH GMBH,nan,nan,nan,GB57METR12345678901234,BODENSEE STR. 226 22761 HAMBURG,DE,nan,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,NORDFISCH GMBH,,True
1,210322,MT103 0001,PAYMENT FOR GOODS,USD,10000,CRED,o,nan,CRED,ABC INDUSTRIES,nan,nan,nan,"123 MAIN STREET NEW YORK, NY 10001",US,US12345678901234567890,nan,ABCBUS33XXX,ABC INDUSTRIES,US,nan,HBUKGB4BXXX,XYZ SUPPLIERS,nan,nan,nan,CN123456789012345678,123 HUANGPU ROAD SHANGHAI,CN,nan,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,XYZ SUPPLIERS,,True
2,210322,MT103 0001,INV NO. 12345REF. 98765SUPPLY OF GOODS AS PER ...,USD,9899,CRED,o,nan,CRED,ABC SUPPLIERS BV,nan,nan,nan,AMSTERDAM,NL,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,nan,ABNANL2A,ABC SUPPLIERS BV,NL,nan,HBUKGB4BXXX,AFRICAN EXPORT-IMPORT BANK,nan,nan,nan,PASSNGLAXXX,"LAGOS, NIGERIA XYZ ENTERPRISES LTD LAGOS",NG,nan,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,AFRICAN EXPORT-IMPORT BANK,,False


<h3>High risk countries</h3>
Financial secrecy and money laundering are closely linked, as financial secrecy can provide a safe haven for criminals to hide and launder illicitly obtained funds. When a country has weak or non-existent regulations around financial transparency, it becomes an attractive destination for criminals looking to hide their illicit activities.

The detection is done by looking at the originator country of the funds, if this country is included in the list of high financial secrecy, the transaction is flagged with high_risk_country = True.

In [ ]:
# TODO: import text file instead of premade list
def high_risk_country(df, path_to_countries_file):
    # List of country codes of highest risk countries based on 
    # https://fsi.taxjustice.no/fsi/2022/world/index/top from text file
    countries_file = open(path_to_countries_file, "r")
    risk_countries = countries_file.read()
    high_risk_countries = risk_countries.split('\n')
    high_risk_countries = [item.strip() for item in high_risk_countries]

    for index, country_iso in df['originator_country'].iteritems():
        if country_iso in high_risk_countries:
        # set the value to True for the corresponding row
            df.at[index, 'high_risk_country'] = True
        else:
            df.at[index, 'high_risk_country'] = False

high_risk_country(df, "high_risk_countries.txt")
df



,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,transaction_direction,transaction_status,instrument_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_account_number,beneficiary_address,beneficiary_country,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country,round_number,high_risk_country
0,210322,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,o,nan,CRED,COMMERZBANK AG,nan,nan,nan,HAMBURG,DE,DE98765432101234567890,nan,COBADEHHXXX,COMMERZBANK AG,DE,nan,HBUKGB4BXXX,NORDFISCH GMBH,nan,nan,nan,GB57METR12345678901234,BODENSEE STR. 226 22761 HAMBURG,DE,nan,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,NORDFISCH GMBH,,True,True
1,210322,MT103 0001,PAYMENT FOR GOODS,USD,10000,CRED,o,nan,CRED,ABC INDUSTRIES,nan,nan,nan,"123 MAIN STREET NEW YORK, NY 10001",US,US12345678901234567890,nan,ABCBUS33XXX,ABC INDUSTRIES,US,nan,HBUKGB4BXXX,XYZ SUPPLIERS,nan,nan,nan,CN123456789012345678,123 HUANGPU ROAD SHANGHAI,CN,nan,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,XYZ SUPPLIERS,,True,True
2,210322,MT103 0001,INV NO. 12345REF. 98765SUPPLY OF GOODS AS PER ...,USD,9899,CRED,o,nan,CRED,ABC SUPPLIERS BV,nan,nan,nan,AMSTERDAM,NL,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,nan,ABNANL2A,ABC SUPPLIERS BV,NL,nan,HBUKGB4BXXX,AFRICAN EXPORT-IMPORT BANK,nan,nan,nan,PASSNGLAXXX,"LAGOS, NIGERIA XYZ ENTERPRISES LTD LAGOS",NG,nan,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,AFRICAN EXPORT-IMPORT BANK,,False,False


<h3>Nesting</h3>
In banking, "nesting" (also known as "layering") is a method of money laundering that involves moving funds through multiple accounts or layers of financial transactions in order to obscure the source and ownership of the funds.

Detection of nesting is done by detecting obscure locations by any involved party, such as beneficiary, intermediary or originator. 

In [ ]:
def nesting_detection(df, path_to_countries_file):
    # Read the countries from a .txt file
    countries_file = open(path_to_countries_file, "r")
    risk_countries = countries_file.read()
    nesting_countries = risk_countries.split('\n')


    for index, row in df.iterrows():
        # Detect as many layers as possible which could facilitate nesting
        if (row['originator_fi_country'] in nesting_countries) or (row['beneficiary_country'] in nesting_countries) or (row['beneficiary_fi_country'] in nesting_countries) or (row['originator_country'] in nesting_countries) or (str(row['outgoing_intermediary_fi_bic']) and row['outgoing_intermediary_fi_bic'][4:6] in nesting_countries) or (str(row['incoming_intermediary_fi_bic']) and row['incoming_intermediary_fi_bic'][4:6] in nesting_countries):
            df.at[index, 'nesting_detection'] = True
        else:
            df.at[index, 'nesting_detection'] = False

nesting_detection(df, 'nesting_countries.txt')
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,transaction_direction,transaction_status,instrument_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_account_number,beneficiary_address,beneficiary_country,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country,round_number,high_risk_country,nesting_detection
0,210322,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,o,nan,CRED,COMMERZBANK AG,nan,nan,nan,HAMBURG,DE,DE98765432101234567890,nan,COBADEHHXXX,COMMERZBANK AG,DE,nan,HBUKGB4BXXX,NORDFISCH GMBH,nan,nan,nan,GB57METR12345678901234,BODENSEE STR. 226 22761 HAMBURG,DE,nan,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,NORDFISCH GMBH,,True,True,False
1,210322,MT103 0001,PAYMENT FOR GOODS,USD,10000,CRED,o,nan,CRED,ABC INDUSTRIES,nan,nan,nan,"123 MAIN STREET NEW YORK, NY 10001",US,US12345678901234567890,nan,ABCBUS33XXX,ABC INDUSTRIES,US,nan,HBUKGB4BXXX,XYZ SUPPLIERS,nan,nan,nan,CN123456789012345678,123 HUANGPU ROAD SHANGHAI,CN,nan,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,XYZ SUPPLIERS,,True,True,True
2,210322,MT103 0001,INV NO. 12345REF. 98765SUPPLY OF GOODS AS PER ...,USD,9899,CRED,o,nan,CRED,ABC SUPPLIERS BV,nan,nan,nan,AMSTERDAM,NL,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,nan,ABNANL2A,ABC SUPPLIERS BV,NL,nan,HBUKGB4BXXX,AFRICAN EXPORT-IMPORT BANK,nan,nan,nan,PASSNGLAXXX,"LAGOS, NIGERIA XYZ ENTERPRISES LTD LAGOS",NG,nan,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,AFRICAN EXPORT-IMPORT BANK,,False,False,False


<h3>Smurfing</h3>
In banking, "smurfing" (also known as "structuring") refers to the illegal practice of making multiple small deposits or transactions instead of one large deposit or transaction in order to avoid suspicion and detection by authorities.

The detection is done by analyzing multiple transactions from the same originator in a given timeframe, with a self imposed threshold.

In [ ]:
def detect_smurfing(df, threshold=10000):
    # Group transactions by 'originator_account_number' and 'transaction_date'
    grouped_transactions = df.groupby(['originator_account_number', 'transaction_date'])['transaction_amount'].sum().reset_index()
    # Create a smurfing_detection column in the grouped_transactions dataframe
    grouped_transactions['smurfing_detection'] = int(grouped_transactions['transaction_amount'][0]) > threshold
    # Merge the 'smurfing_detection' column back into the original dataframe
    df = pd.merge(df, grouped_transactions[['originator_account_number', 'transaction_date', 'smurfing_detection']], on=['originator_account_number', 'transaction_date'], how='left')

detect_smurfing(df)
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,transaction_direction,transaction_status,instrument_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_account_number,beneficiary_address,beneficiary_country,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country,round_number,high_risk_country,nesting_detection
0,210322,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,o,nan,CRED,COMMERZBANK AG,nan,nan,nan,HAMBURG,DE,DE98765432101234567890,nan,COBADEHHXXX,COMMERZBANK AG,DE,nan,HBUKGB4BXXX,NORDFISCH GMBH,nan,nan,nan,GB57METR12345678901234,BODENSEE STR. 226 22761 HAMBURG,DE,nan,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,NORDFISCH GMBH,,True,True,False
1,210322,MT103 0001,PAYMENT FOR GOODS,USD,10000,CRED,o,nan,CRED,ABC INDUSTRIES,nan,nan,nan,"123 MAIN STREET NEW YORK, NY 10001",US,US12345678901234567890,nan,ABCBUS33XXX,ABC INDUSTRIES,US,nan,HBUKGB4BXXX,XYZ SUPPLIERS,nan,nan,nan,CN123456789012345678,123 HUANGPU ROAD SHANGHAI,CN,nan,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,XYZ SUPPLIERS,,True,True,True
2,210322,MT103 0001,INV NO. 12345REF. 98765SUPPLY OF GOODS AS PER ...,USD,9899,CRED,o,nan,CRED,ABC SUPPLIERS BV,nan,nan,nan,AMSTERDAM,NL,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,nan,ABNANL2A,ABC SUPPLIERS BV,NL,nan,HBUKGB4BXXX,AFRICAN EXPORT-IMPORT BANK,nan,nan,nan,PASSNGLAXXX,"LAGOS, NIGERIA XYZ ENTERPRISES LTD LAGOS",NG,nan,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,AFRICAN EXPORT-IMPORT BANK,,False,False,False


<h3>Non-adherence to FATF</h3>
In banking, "smurfing" (also known as "structuring") refers to the illegal practice of making multiple small deposits or transactions instead of one large deposit or transaction in order to avoid suspicion and detection by authorities.

The detection is done by analyzing multiple transactions from the same originator in a given timeframe, with a self imposed threshold.

In [ ]:
def check_fatf_adherence(df):
    required_fields = [
        'originator_full_name',
        'originator_account_number',
        'originator_address',
        'beneficiary_address',
        'beneficiary_full_name',
        'beneficiary_account_number'
    ]

    # Check each row with a nested function, check on values in the required fields
    # Set fatf_adherence to false if one of the fields is empty.
    def is_adherent(row):
        for field in required_fields:
            if pd.isnull(row[field]) or row[field] == '':
                return False
        return True

    df['fatf_adherence'] = df.apply(is_adherent, axis=1)

check_fatf_adherence(df)
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,transaction_direction,transaction_status,instrument_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_account_number,beneficiary_address,beneficiary_country,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country,round_number,high_risk_country,nesting_detection,fatf_adherence
0,210322,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,o,nan,CRED,COMMERZBANK AG,nan,nan,nan,HAMBURG,DE,DE98765432101234567890,nan,COBADEHHXXX,COMMERZBANK AG,DE,nan,HBUKGB4BXXX,NORDFISCH GMBH,nan,nan,nan,GB57METR12345678901234,BODENSEE STR. 226 22761 HAMBURG,DE,nan,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,NORDFISCH GMBH,,True,True,False,True
1,210322,MT103 0001,PAYMENT FOR GOODS,USD,10000,CRED,o,nan,CRED,ABC INDUSTRIES,nan,nan,nan,"123 MAIN STREET NEW YORK, NY 10001",US,US12345678901234567890,nan,ABCBUS33XXX,ABC INDUSTRIES,US,nan,HBUKGB4BXXX,XYZ SUPPLIERS,nan,nan,nan,CN123456789012345678,123 HUANGPU ROAD SHANGHAI,CN,nan,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,XYZ SUPPLIERS,,True,True,True,True
2,210322,MT103 0001,INV NO. 12345REF. 98765SUPPLY OF GOODS AS PER ...,USD,9899,CRED,o,nan,CRED,ABC SUPPLIERS BV,nan,nan,nan,AMSTERDAM,NL,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,nan,ABNANL2A,ABC SUPPLIERS BV,NL,nan,HBUKGB4BXXX,AFRICAN EXPORT-IMPORT BANK,nan,nan,nan,PASSNGLAXXX,"LAGOS, NIGERIA XYZ ENTERPRISES LTD LAGOS",NG,nan,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,AFRICAN EXPORT-IMPORT BANK,,False,False,False,True


<h3>Trade Based Money-Laundering</h3>
According to the FATF/Egmont report: Trade-Based Money Laundering (TBML) risk 

- indicators include: payments routed in a circle - funds are sent from one country and received back in the same country, after passing through another country or countries (intermediaries).
- Further, contracts, invoices or other trade related data or documents have vague descriptions of traded commodities. (transaction_message)
- Further, contracts, invoices or other documents display fees out of line with commercial consideration or market value. However, this data is out of the scope of the available data.

Another check for TBML is the 'transaction_type', which should be either 'precious_metal', 'interest/dividend', 'crypto/virtual_assets' or 'securities'

In [ ]:
def detect_tbml(df):
    vague_descriptions = [
        'goods',
        'products',
        'commodities',
        'supplies',
        'merchandise',
        'stuff',
    ]

    risky_transaction_types = [
        'precious_metal',
        'interest/dividend',
        'crypto/virtual_assets',
        'securities',
    ]

    # Check for inclusion of vague words, as of the list above
    def is_vague_message(message):
        message_lower = message.lower()
        for desc in vague_descriptions:
            if desc in message_lower:
                return True
        return False

    # If the transaction type isn't associated with trading, it cannot be TBML
    def is_tbml(row):
        if row['transaction_type'] not in risky_transaction_types:
            return False

        # If the originator and beneficiary country is identical, but the transaction
        # is routed through a foreign country, this will set of alerts.
        # use the bic of intermediaries to check country
        if row['originator_country'] == row['beneficiary_country']:
            if (row['incoming_intermediary_fi_bic'][:2] != row['originator_country']
                    or row['outgoing_intermediary_fi_bic'][:2] != row['beneficiary_country']):
                return True

        # Another trigger is the vague description of trading documents, which 
        # will be derived from the transaction message
        if is_vague_message(row['transaction_message']):
            return True

        return False

    df['tbml_detection'] = df.apply(is_tbml, axis=1)

detect_tbml(df)
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,transaction_direction,transaction_status,instrument_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_account_number,beneficiary_address,beneficiary_country,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country,round_number,high_risk_country,nesting_detection,fatf_adherence,tbml_detection
0,210322,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,o,nan,CRED,COMMERZBANK AG,nan,nan,nan,HAMBURG,DE,DE98765432101234567890,nan,COBADEHHXXX,COMMERZBANK AG,DE,nan,HBUKGB4BXXX,NORDFISCH GMBH,nan,nan,nan,GB57METR12345678901234,BODENSEE STR. 226 22761 HAMBURG,DE,nan,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,NORDFISCH GMBH,,True,True,False,True,False
1,210322,MT103 0001,PAYMENT FOR GOODS,USD,10000,CRED,o,nan,CRED,ABC INDUSTRIES,nan,nan,nan,"123 MAIN STREET NEW YORK, NY 10001",US,US12345678901234567890,nan,ABCBUS33XXX,ABC INDUSTRIES,US,nan,HBUKGB4BXXX,XYZ SUPPLIERS,nan,nan,nan,CN123456789012345678,123 HUANGPU ROAD SHANGHAI,CN,nan,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,XYZ SUPPLIERS,,True,True,True,True,False
2,210322,MT103 0001,INV NO. 12345REF. 98765SUPPLY OF GOODS AS PER ...,USD,9899,CRED,o,nan,CRED,ABC SUPPLIERS BV,nan,nan,nan,AMSTERDAM,NL,/NL20ABNA0404875234\nABNANL2A\nABC SUPPLIERS B...,nan,ABNANL2A,ABC SUPPLIERS BV,NL,nan,HBUKGB4BXXX,AFRICAN EXPORT-IMPORT BANK,nan,nan,nan,PASSNGLAXXX,"LAGOS, NIGERIA XYZ ENTERPRISES LTD LAGOS",NG,nan,/PASSNGLAXXX\nAFRICAN EXPORT-IMPORT BANK\nLAGO...,AFRICAN EXPORT-IMPORT BANK,,False,False,False,True,False
